# Initial Comments

I have been working with the things we discussed on Friday. I dropped all REITS that contianed missing values, reducing the dataset from 203 to 159 securities. I have tried implementing the control-variables you suggested, such as the return of the index, named "BBREIT INDEX" in the dataset and the squared returns named "VOL" in the dataset.

The quoted spread is calculated the same way as they are in Chordia.

I also figured out how to extract the specifics from the regression tables that I needed and managed to produce means and medians of them. 

In [1]:
# IMporting relevant packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.stats.stattools import durbin_watson

In [2]:
# Importing Dataset and making necessary ammendments

df=pd.read_excel("BBREIT-1.05.xlsx",sheet_name="PROPORTIONAL-QUOTED-SPREAD", index_col="Dates")
df=df.dropna(axis=1)
df2=df.copy()
df3=df2.copy()
df3=df3.drop(columns="BBREIT Index") # Dropping Index Variable to avoid it distorting the sum-variable thas will be generated in line 5
df2["BBREIT Index"]=np.log(df["BBREIT Index"])-np.log(df["BBREIT Index"].shift(1)) # Transforming Index-Variable in df2 then will add it to Data Frame 3 after sum generaiton
df2=df2.dropna(axis=0)

In [3]:
# Transforming the data to continous form

for i in df3:
    df3[i]=np.log(df3[i])-np.log(df3[i].shift(1))

In [4]:
df3=df3.dropna(axis=0)

In [5]:
# Figured out how to generate the sum-variable in Python

df3["Sum"]=df3.sum(axis=1)

In [6]:
df3["BBREIT Index"]=df2["BBREIT Index"] # Placing it back in

In [7]:


# Generating empty lists to store the coefficients, r^2s, p-values and more
Coefficients = []
Pvalues = []
Rsquared_adj = []
Bpagan = []
DBWatson = []
STD = []

for i in df3:
    
    df3["1lag"]=df3[i].shift(1) # Lag-variables to try to correct for autocorrelation
    df3["2lag"]=df3[i].shift(2)
    df3["3lag"]=df3[i].shift(3)
    df3["4lag"]=df3[i].shift(4)
    df3["VOL"]= ((df3[i]-df3[i].shift(1))/df3[i].shift(1))**2 # Squared return variable as we spoke about
    df3["Index"]=((df3["Sum"]-df3[i])/157) # Cross Sectional Mean Variable
    df3.replace([np.inf], np.nan, inplace=True)
    df3=df3.fillna(0)
        
        
      
    y = df3[i]
    x = df3[["Index","BBREIT Index","VOL","1lag","2lag","3lag","4lag"]]
    x = sm.add_constant(x)
    model = sm.OLS(y,x, missing = 'drop') 
    results = model.fit()
    print(results.summary())
    print(sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x))
    print(durbin_watson(results.resid))
    
    
    # Filling the lists with the content specified
    coefficients=results.params 
    Coefficients.append(coefficients)
    
    pvalues=results.pvalues
    Pvalues.append(pvalues)
    
    rsquared_adj = results.rsquared_adj
    Rsquared_adj.append(rsquared_adj)
    
    bpagan = sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x)
    Bpagan.append(bpagan)
    
    std=results.bse
    STD.append(std)
    
    dbwatson = durbin_watson(results.resid)
    DBWatson.append(dbwatson)
    
    # CLearing the dynamic variables after use. Mostly for practical reuse-reasons
    Drop=df3[["Index", "VOL","1lag","2lag","3lag","4lag"]]
    df3.drop(columns=Drop, inplace=True)
    

                            OLS Regression Results                            
Dep. Variable:          AMT US Equity   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.393
Method:                 Least Squares   F-statistic:                     52.61
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           2.29e-57
Time:                        16:19:06   Log-Likelihood:                 291.64
No. Observations:                 559   AIC:                            -567.3
Df Residuals:                     551   BIC:                            -532.7
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0021      0.006      0.345   

(18.69393728199549, 0.00920214586571443, 2.7234192282382814, 0.008749466008403097)
2.1771502366024795
                            OLS Regression Results                            
Dep. Variable:         VICI US Equity   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     41.12
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           1.40e-46
Time:                        16:19:06   Log-Likelihood:                 776.17
No. Observations:                 559   AIC:                            -1536.
Df Residuals:                     551   BIC:                            -1502.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

(35.75030119700178, 8.079588716171605e-06, 5.378043081974241, 5.632693262132521e-06)
2.3258024101385755
                            OLS Regression Results                            
Dep. Variable:          SUI US Equity   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     78.31
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           1.60e-78
Time:                        16:19:06   Log-Likelihood:                 274.23
No. Observations:                 559   AIC:                            -532.5
Df Residuals:                     551   BIC:                            -497.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------

                            OLS Regression Results                            
Dep. Variable:          AMH US Equity   R-squared:                       0.377
Model:                            OLS   Adj. R-squared:                  0.369
Method:                 Least Squares   F-statistic:                     47.67
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           7.55e-53
Time:                        16:19:06   Log-Likelihood:                 348.25
No. Observations:                 559   AIC:                            -680.5
Df Residuals:                     551   BIC:                            -645.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0003      0.006      0.050   

(35.89309672020551, 7.59387169479642e-06, 5.400998252346863, 5.273983727041171e-06)
1.9266375605548178
                            OLS Regression Results                            
Dep. Variable:          NNN US Equity   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     50.01
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           5.16e-55
Time:                        16:19:07   Log-Likelihood:                 266.47
No. Observations:                 559   AIC:                            -516.9
Df Residuals:                     551   BIC:                            -482.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

...

                            OLS Regression Results                            
Dep. Variable:          BRT US Equity   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.380
Method:                 Least Squares   F-statistic:                     49.87
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           6.97e-55
Time:                        16:19:10   Log-Likelihood:                -369.10
No. Observations:                 559   AIC:                             754.2
Df Residuals:                     551   BIC:                             788.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0027      0.020     -0.133   

(3.742490434200926, 0.8089136819173319, 0.5305420570557141, 0.8116141446247135)
2.160921978640669
                            OLS Regression Results                            
Dep. Variable:          FSP US Equity   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     42.05
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           1.73e-47
Time:                        16:19:11   Log-Likelihood:                 445.70
No. Observations:                 559   AIC:                            -875.4
Df Residuals:                     551   BIC:                            -840.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

(56.51348825265156, 7.467954035739715e-10, 8.852812477614968, 2.4011994393752694e-10)
2.0991756279401836
                            OLS Regression Results                            
Dep. Variable:         CLPR US Equity   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     41.14
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           1.35e-46
Time:                        16:19:11   Log-Likelihood:                -54.810
No. Observations:                 559   AIC:                             125.6
Df Residuals:                     551   BIC:                             160.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------

                            OLS Regression Results                            
Dep. Variable:         CORR US Equity   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     37.34
Date:                Tue, 18 Oct 2022   Prob (F-statistic):           8.11e-43
Time:                        16:19:11   Log-Likelihood:                 73.354
No. Observations:                 559   AIC:                            -130.7
Df Residuals:                     551   BIC:                            -96.10
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0026      0.009      0.281   

(45.52980514551479, 1.0790745864336472e-07, 8.157712200947575, 1.8160654066043597e-08)
2.146735488752583
                            OLS Regression Results                            
Dep. Variable:           BBREIT Index   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.627e+27
Date:                Tue, 18 Oct 2022   Prob (F-statistic):               0.00
Time:                        16:19:12   Log-Likelihood:                 17988.
No. Observations:                 559   AIC:                        -3.596e+04
Df Residuals:                     551   BIC:                        -3.592e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------

In [8]:
# Turning the generated lists in the loop into dataframes

Pvalsum=pd.DataFrame(Pvalues)
RsquaredSum=pd.DataFrame(Rsquared_adj)
BpaganSum=pd.DataFrame(Bpagan)
DBWatsonSum=pd.DataFrame(DBWatson)
CoefficientsSum=pd.DataFrame(Coefficients)
STDsum=pd.DataFrame(STD)

In [9]:
# Dropping the last two rows as they are uninteresting (Sum and BBREIT Index)

Pvalsum.drop(Pvalsum.index[158:160], inplace = True)
RsquaredSum.drop(RsquaredSum.index[158:160], inplace = True)
BpaganSum.drop(BpaganSum.index[158:160], inplace = True)
DBWatsonSum.drop(DBWatsonSum.index[158:160], inplace = True)
CoefficientsSum.drop(CoefficientsSum.index[158:160], inplace = True)

In [10]:
# Average Cross-sectional-Average-coefficient. Coefficient resembles what we see in Chordia with a mean of 0.69

round(CoefficientsSum ["Index"].mean(),4)

0.8004

In [11]:
# Median Cross-sectional-Average-coefficient somewhat lower than the mean, opposite of Chordia who had a higher median than mean coefficient

round(CoefficientsSum ["Index"].median(),4)

0.7497

In [12]:
# Average P-value also in the range of Chordia

round(Pvalsum["Index"].mean(),4)

0.0083

In [13]:
# Average adjusted r^2. This I see as the largest problem thus far. Average R^2 definately too high. Will be the main 
# area of focus to get this number down.

round(RsquaredSum.mean(),4)

0    0.3817
dtype: float64

In [14]:
# Average DB Watson test-value. I don`t excactly remember the rules for this test but I think this is acceptable.
# Will at any rate repeat some statistics in the near future.

round(DBWatsonSum.mean(),4)

0    2.1735
dtype: float64

In [15]:
# Average P-Value for Breuch Pagan. I suppose this means my models are homoscedastic on the normal significance leveles (1,5,10)

round(BpaganSum[1].mean(),4)

0.1533

In [16]:
# Average standard deviation

round(STDsum["Index"].mean(),4)

0.0727

# Final Comments

I am working on making a table in python that can make the relevant outpurs a little more readable. What I might need help with now is some guidance as to what to do with the statistics within the models and if there are any obvious mistakes that you might see in the code/maths. Things are progressing quite well and quick.